# Threat Hunting in Apache Logs

## Background
The following is interactive Jupyter notebook code that parsed Apache access logs and ingested them into Pandas data frames for statistical analysis. The outputs from executed notebook cells will be saved to CSV and PNG files respectively and can be found in the images_folder or output_folder.

### Directions

This project will be initialized with the following directory schema:
    
```
project_name_folder (user creates this)
|  
+-- apache_accessLogs_hunt.ipynb (user would start by placing this notebook into an empty project folder) 
|  
+-- project_README.md (user would document external project dependencies, etc.) 
|
+-- data_logs_folder (this gets created, user can then upload/move log files to this dir)
|    |
|    +-- original_log.csv/.log/.gz etc.
|    |
|    +-- output_folder (this gets created, used to store files that are outputs of Pandas DF manipulation)
|    |
|    \-- images_folder (this gets created, used to store image files that are saved graphs/visualizations)
|
+-- max_mind_db (this gets created, used to store GeoIP database)
     | 
     +-- GeoLite2-ASN.mmdb
     |
     \-- GeoLite2-City.mmdb
```

## Arrange the workbench and tools required

In [ ]:
#Check to make sure conda env path is appropriate for Jupy NB context (risk_assessment)
#Sys.executable needs to be something like: /Users/bpalm/opt/anaconda3/envs/risk_assessment/bin/python
import sys

#Then do check to see if risk_assessment exists
if 'threat_hunt' not in sys.executable:
    print('Error: Conda Env is not set to the correct path! Trouble shoot conda env path.')

#Check python version
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

In [ ]:
#File upload section where the directory structure gets built and then analyst could upload their file(s) to be worked on
from pathlib import Path
from IPython.display import Markdown, display

#Check for .env files and msticpyconfig.yaml
dot_env = Path.cwd() / '.env'
if not dot_env.is_file():
    print('Please create a .env file in the current working directory. This file contains API keys for HIBP, Shodan, etc pivots.')

mstic_config = Path.cwd() / 'msticpyconfig.yaml'
if not mstic_config.is_file():
    print('Please create a msticpyconfig.yaml file in the current working directory.')

#Print Markdown in output
def printmd(string):
    display(Markdown(string))

#Make data_logs_folder
data_logs_folder = Path.cwd() / 'data_logs_folder'
if Path(data_logs_folder).exists() != True:
    data_logs_folder.mkdir()
    printmd('**data_logs_folder created**')

else:
    printmd('**data_logs_folder already present**')

#Make images_folder
images_folder = Path.cwd() / 'images_folder'
if Path(images_folder).exists() != True:
    images_folder.mkdir()
    printmd('**images_folder created**')

else:
    printmd('**images_folder already present**')

#Make output_folder
output_folder = Path.cwd() / 'output_folder'
if Path(output_folder).exists() != True:
    output_folder.mkdir()
    printmd('**output_folder created**')

else:
    printmd('**output_folder already present**')

#Make max_mind_db_folder - this will get created with the msticpy bootstrapping
#This module import addresses the MSTICpy / MaxMind db bootstrap chore
import msticpy.sectools as sectools
from msticpy.nbtools import *
from msticpy.nbtools.entityschema import IpAddress, GeoLocation
from msticpy.sectools.geoip import GeoLiteLookup, IPStackLookup

#Quick check on dir path and file existing
max_mind_path = Path.cwd()/'max_mind_db'
max_mind_db = Path(max_mind_path/'GeoLite2-City.mmdb')

if Path(max_mind_path).exists() == True:
    if max_mind_db.is_file():
        printmd('**max_mind_db dir and mmdb already present**')

#This initiates the maxmind db pull and dir creation or the db update process
iplocation = GeoLiteLookup()

In [ ]:
from dotenv import load_dotenv
import geoip2.database
import gzip
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from ipywidgets import Layout, Box, Dropdown, RadioButtons, Output
from ipwhois import IPWhois
import json
import kaleido
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import pycountry
import requests
from user_agents import parse
import whois

#Doesn't truncate Pandas DF if you uncomment below
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

#This function allows pio.renderers.default = w.value to be set during an on_change event and not in the following NB cell
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        pio.renderers.default = selecta.value

def check_file(change):
    if fc.selected_filename == None:
        #raise alert
        caption.value = 'File needs to be selected in FileChooser drop down.'

    elif change.new not in fc.selected_filename:
        #raise alert
        caption.value = 'File type does not match the file selected.'
    
    elif change.new in fc.selected_filename:
        caption.value = 'File type matches file selected, please continue!'

#Use drop down widget to establish path to directory that holds the log files
# Create and display a FileChooser widget
fc = FileChooser()
fc.use_dir_icons = True
fc.default_path = str(Path.cwd()/'data_logs_folder')

caption = widgets.Label(value='The log type has not been selected...')

#Use radio button widget to establish file type
selecta_ext = RadioButtons(
    options=['.csv','.txt','.json','.log'],
    value='.csv', # Defaults to '.csv'
    style={'description_width':'initial'},
    layout={'width': 'max-content'}, # If the items' names are long
    description='Select log file type to be parsed:',
    disabled=False
)
selecta_ext.observe(check_file,names='value')

#Fix for plotly graph rendering in Firefox browser - notebooks, labs, and PDF export
selecta = Dropdown(
    options=['notebook', 'plotly_mimetype', 'notebook+plotly_mimetype+pdf'],
    value='notebook',
    description='Plotly pio.renderers setting:',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)
selecta.observe(on_change)

#Brad TODO: make these instructions for selections in markdown!
print("Please navigate to the directory that contains the logs to be analyzed.")
print("Select a log file in the directory with the File Chooser.")
print("Select the file type of the logs to be imported.")
print("The selected path and file type will be stored and used to analyze all file extensions of that type in the directory.")
display(fc, caption, selecta_ext, selecta)

## Load Materials onto the workbench

Logs were provided in advance and were downloaded as a zip file. The files were unzipped and placed into the data_logs_folder directory.

In [ ]:
#Testing for Apache logs
#The below did not work and got hung up on a few logs' timezones



# from datetime import datetime
# import matplotlib.pyplot as plt
# import pandas as pd
# from pandas import DataFrame, Series
# import pytz
# import re

# def parse_str(x):
#     """
#     Returns the string delimited by two characters.

#     Example:
#         `>>> parse_str('[my string]')`
#         `'my string'`
#     """
#     return x[1:-1]

# def parse_datetime(x):
#     '''
#     Parses datetime with timezone formatted as:
#         `[day/month/year:hour:minute:second zone]`

#     Example:
#         `>>> parse_datetime('13/Nov/2015:11:45:42 +0000')`
#         `datetime.datetime(2015, 11, 3, 11, 45, 4, tzinfo=<UTC>)`

#     Due to problems parsing the timezone (`%z`) with `datetime.strptime`, the
#     timezone will be obtained using the `pytz` library.
#     '''
#     dt = datetime.strptime(x[1:-7], '%d/%b/%Y:%H:%M:%S')
#     dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
#     return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))


# def parse_datetime(x):
#     dt = datetime.strptime(x[1:-7], '%d/%b/%Y:%H:%M:%S') %m/%d/%Y %H:%M:%S
#     dt_tz = int(x[-6:-3])*60+int(x[-3:-1])
#     return dt.replace(tzinfo=pytz.FixedOffset(dt_tz))

In [ ]:

# apache_log_path = Path.cwd()/'data_logs_folder/help1.20211209-1211/local1/apache2/logs/help/access_2021_11_12_21_49_52.log'

# data = pd.read_csv(
#     apache_log_path,
#     sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
#     engine='python',
#     na_values='-',
#     header=None,
#     usecols=[0, 3, 4, 5, 6, 7, 8],
#     names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
#     converters={'time': parse_datetime,
#                 'request': parse_str,
#                 'status': int,
#                 'size': int,
#                 'referer': parse_str,
#                 'user_agent': parse_str})

In [ ]:
def parseApacheLogs(filename):
    fields = ['host', 'identity', 'user', 'time_part1', 'time_part2', 'cmd_path_proto',
             'http_code', 'response_bytes', 'referer', 'user_agent']
    
    data = pd.read_csv(filename, sep=' ', header=None, names=fields, na_values=['-'], dtype={"cmd_path_proto": "string", "http_code": "string", "user_agent": "string"})

    # Panda's parser mistakenly splits the date into two columns, so we must concatenate them
    time = data.time_part1 + data.time_part2
    time_trimmed = time.map(lambda s: s.strip('[]').split('-')[0]) # Drop the timezone for simplicity
    data['time'] = pd.to_datetime(time_trimmed, format='%d/%b/%Y:%H:%M:%S')
    
    # # Split column `cmd_path_proto` into three columns, and decode the URL (ex: '%20' => ' ')
    # data['command'], data['path'], data['protocol'] = zip(*data['cmd_path_proto'].str.split().tolist())
    # data['path'] = data['path'].map(lambda s: unquote(s))
    request = data.pop('cmd_path_proto').str.split()
    data['method'] = request.str[0]
    data['resource'] = request.str[1]
    data['protocol'] = request.str[2]
    data=data[['time','time_part1', 'time_part2','host','identity', 'user','method','resource','protocol','http_code','response_bytes','referer','user_agent']]
    
    # # Drop the fixed columns and any empty ones
    data1 = data.drop(['time_part1', 'time_part2'], axis=1)
    return data1.dropna(axis=1, how='all')

In [ ]:
#Now what paths do we need to move forward
# - data_logs_folder holds all the log files and can have subdirectories with dates
# - images_folder is a subdir of data_logs_folder
# - output_folder is a subdir of data_logs_folder
# - max_mind_db is at same level/hierarchy as data_logs_folder

#Working folder will be the selected_path from File Chooser fc var
working_folder = Path(fc.selected_path)

#Establish array of log files to be processed using Pathlib methods iterdir and is_file
# Had to wrap the iterdir with sorted since it wasn't loading them by time stamp!
# Make sure there are no hidden files or non-log file types,
# l only gets added if is_file and endswith .log and has access in filename
file_list = [l for l in sorted(working_folder.iterdir()) if l.is_file() and (l.name.endswith(selecta_ext.value) and "access" in l.name)]
assert (len(file_list) > 0), "No log files were found in dir!"

# crate a list to add dataframes to
apache_list = list()

completedFiles = 0
eventsProcessed = 0
print("Number of logs to be analyzed: ",len(file_list))

for log in file_list:
    try:
        temp = parseApacheLogs(log)
    except ValueError as e:
        print(e)
        print(f"The error occurred in this log file: {log}")
    
    eventsProcessed += len(temp)

    # normalize the file and append it to the list of dataframe 
    apache_list.append(temp)
    
    #Print out finished analysis of log name
    completedFiles += 1

# concat the files into a single dataframe
apache = pd.concat(apache_list).reset_index(drop=True)

print("Number of logs analyzed:",completedFiles) 
print("Number of events processed:", eventsProcessed)

#Now we have apache df loaded onto the workbench, time to hunt!

*TODO:* Better option for parsing logs moving forward, as it handles the time format issue!

In [ ]:
# from apachelogs import LogParser
# parser = LogParser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")

# apache_log_path = Path.cwd()/'data_logs_folder/help1.20211209-1211/local1/apache2/logs/help/access_2021_11_12_21_49_52.log'

# with open(apache_log_path) as fp:
#     for entry in parser.parse_lines(fp):
#         print(str(entry.request_time), entry.request_line)

## STOP! - Determine if the right materials have been loaded on the workbench - do timestamps seem correct, do number of rows/events seem correct?
There was an issue with loading multiple CSV files (read_csv) into Pandas DF that lead me to question if read_json behaves in the same way.

* [X] dataframe that was loaded checks out

In [ ]:
display(apache.head())

In [ ]:
display(apache.tail())

### Data Cleaning and Manipulation

In [ ]:
h1_key_str = 'HTTP/1.1"'
h1_boolean_findings = apache['http_code'].str.contains(h1_key_str)
total_occurence = h1_boolean_findings.sum()
if(total_occurence > 0):
    print(f"Yes, {h1_key_str} is present in the data frame")
elif(total_occurence == 0):
    print(f"No, {h1_key_str} is not present in the data frame")

In [ ]:
#show the rows that contain key_str for further inspection
showString_df=apache[apache.http_code.isin([h1_key_str])]
display(showString_df)
showString_df.shape

In [ ]:
apache_drop = apache[apache['http_code'] != 'HTTP/1.1"']

In [ ]:
len(apache_drop)

In [ ]:
apache_drop = apache_drop.astype({"http_code": int})

In [ ]:
apache_drop.info()

## Objectives
Each threat hunt starts with establishing the following criteria:
* What visibility do I have with regard to identifying attacker behavior? Identifying app misconfigurations? Identifying the efficacy of security controls?
* What vantage point does this data provide and what attacker behaviors can be identified within it?

Once the visibility is established, a hunt is crafted that takes into account the data available and what techniques can be used to identify attacker behavior. A hunt is centered around a well formed hypothesis that seeks to address a single question about the data and attacker.

Apache access logs provide insight into:
* Time
* Host (client IP)
* Request (method, resource, protocol)
* HTTP Code (successful, unsuccessful, etc)
* Referer (redirect, link)
* User-Agent

The following hunts addressed different areas of the Attacker's Kill Chain and could be explored within the data set, given the visibility provided by the CloudTrail logs: 

* Persistance (Hunt 1 - H1)
    - [X] Webshell - POST requests that are valid

### Quick Stack Count of HTTP Codes, Client IP Addresses, Requests, Referer User-Agents 

See what falls out of top talkers/ top events prime the analyst pump!

Column headers:

`['time','host','method','resource','protocol','http_code','response_bytes','referer','user_agent']`

#### HTTP Method Stacking

In [ ]:
apache_drop['method'].value_counts()

#### Client IP Stacking

In [ ]:
apache_drop['host'].value_counts()

In [ ]:
# Our tail looks pretty long....lets see
aggregateIPs = pd.DataFrame(apache_drop.host.value_counts().reset_index().values, columns=["clientIP", "count"])
ip_head_var = 10
print('The top',ip_head_var,'IP talkers from stack counting are shown below:')
display(aggregateIPs.head(ip_head_var))

In [ ]:
#Plotly graphing
fig = px.line(aggregateIPs, x='clientIP', y='count', title='Long Tail Analysis - Most to Least Frequently Occurring IPs')
fig.show()

#### HTTP Status Code Stacking

* 302 - Redirect
* 404 - Not Found, can drop these
* 403 - Forbidden, can drop these or look at these specifically
* 400 - Bad Request, can drop these
* 206 - Partial Content Success
* 304 - Not Modified client redirect, can drop these
* 405 - Method Not Allowed, can drop these
* 401 - Unauthorized, can drop these
* 500 - Internal Server Error, can drop these
* 502 - Bad Gateway, can drop these

In [ ]:
apache_drop['http_code'].value_counts()

## Persistence (H1)

**Hypothesis #1:** If there is an attacker actively exploiting a web shell on the Confluence server, they would be interacting with a web shell that could be identified in Apache Access log data. 

**MITRE ATT&ACK**
- Tactic: Persistence
- Technique/Sub-Technique: T1505.003

**Assumption:** Attackers need to maintain persistent access to target for C2 and additional exploitation. Web shells could be the primary point of origin (patient 0) or it could be secondary or fallback path for persistence.

**Question:** Do the Apache access logs contain evidence of a remote host interacting with a web shell on the Confluence server?

**Method:** An attacker generating a successful request to a web shell will use an HTTP POST method with a lower frequency requested resource.

### Status Code + HTTP Method Filter and then Requested Resource Stacking

In [ ]:
# Now count event + userName that you are interested in
apache_success = apache_drop.copy()
apache_success = apache_success[(apache_success['http_code'] == 200) & (apache_success['method'] == 'POST')] 

In [ ]:
apache_success.head()

In [ ]:
apache_success.tail()

In [ ]:
apache_success.info()

In [ ]:
print(len(apache_success))

#### Requested Resource Stacking
Tried this early in the objective section to get a view of the data but it was too messy and there were too many unique URI strings. This one is more manageable as the dataframe has been filtered down.

In [ ]:
apache_success['resource'].value_counts()

#### Quick Pivot on /rest/tinymce/1/macro/preview String

In [ ]:
h2_key_str = '/rest/tinymce/1/macro/preview'
h2_boolean_findings = apache_success['resource'].str.contains(h2_key_str)
total_occurence = h2_boolean_findings.sum()
if(total_occurence > 0):
    print(f"Yes, {h2_key_str} is present in the data frame")
elif(total_occurence == 0):
    print(f"No, {h2_key_str} is not present in the data frame")

In [ ]:
#show the rows that contain key_str for further inspection
showString_df=apache_success[apache_success.resource.isin([h2_key_str])]
display(showString_df)
showString_df.shape

In [ ]:
#Writeout to CSV file
with open (output_folder/"post_success_lfo.csv", "w") as outfile:
    showString_df.to_csv(outfile, index=False)

#### Quick Pivot on Login String
apache_success is too narrowed for this search, so be sure to use a dataframe prior to the POST + 200 filtering...

In [ ]:
h3_key_str = 'login'
h3_boolean_findings = apache_drop['resource'].str.contains(h3_key_str)
total_occurence = h3_boolean_findings.sum()
if(total_occurence > 0):
    print(f"Yes, {h3_key_str} is present in the data frame")
elif(total_occurence == 0):
    print(f"No, {h3_key_str} is not present in the data frame")

In [ ]:
login_df = apache_drop[(apache_drop['resource'].str.contains(h3_key_str,na=False)) & (apache_drop['http_code'] == 200)]

In [ ]:
login_df.head()

In [ ]:
login_df.tail()

In [ ]:
login_df['resource'].value_counts()

## GeoIP Enrichment Pivot
This is more of an ad-hoc Data-Based Hunt (DBH). This would be more helpful if you had a specific target market and anomalies could stand out based on that. However, still an interesting visualization to help build a baseline.

In [ ]:
geoip_enrich_IPs = apache_drop.copy()
geoip_enrich_IPs = geoip_enrich_IPs['host'].value_counts().rename_axis('source_IP').reset_index(name='counts')
geoip_enrich_IPs.size
# Drop URLs and AWS Internal tags from source_IPs
geoip_enrich_IPs_dropped = pd.DataFrame(geoip_enrich_IPs[geoip_enrich_IPs['source_IP'].str.contains("10.*")==False])

In [ ]:
geoip_enrich_IPs_dropped.info

In [ ]:
apache_drop.head()

In [ ]:
geoip_enrich_IPs = apache_drop.copy()

In [ ]:
geoip_enrich_IPs.head()

In [ ]:
geoip_enrich_IPs = apache_drop.copy()

# Drop internal IPs from host column
geoip_enrich_IPs_dropped = pd.DataFrame(geoip_enrich_IPs[geoip_enrich_IPs['host'].str.contains("10.*|::1|192.168*")==False])
# Drop AWS Health Check-User agent IPs
geoip_enrich_IPs_dropped = geoip_enrich_IPs_dropped[~(geoip_enrich_IPs_dropped['user_agent'] == 'Amazon-Route53-Health-Check-Service (ref afa8566f-3fc3-4314-9bad-cfe2fb9dba91; report http://amzn.to/1vsZADi)')] 
#Stack count user-agents now that items have been dropped
geoip_enrich_IPs_stacked = geoip_enrich_IPs_dropped['host'].value_counts().rename_axis('source_IP').reset_index(name='counts')

In [ ]:
geoip_enrich_IPs_stacked.head(10)

In [ ]:
len(geoip_enrich_IPs_stacked)

In [ ]:
#Create lists to hold the new series data
alpha2 = []
alpha3 = []
country_name = []
lat = []
lon = []
        
reader = geoip2.database.Reader(max_mind_db)

#Was recommended not to do an apply in this multi-column scenario...go figure!
for ip in geoip_enrich_IPs_stacked['source_IP']:
    response = reader.city(ip)
    response_country_checked = ''
    
    if response.country.iso_code == None:
        alpha2.append(response.registered_country.iso_code)
        response_country_checked = response.registered_country.iso_code
        
    elif response.country.iso_code != None:
        alpha2.append(response.country.iso_code)
        response_country_checked = response.country.iso_code

    alpha3.append(pycountry.countries.get(alpha_2=response_country_checked).alpha_3)
    country_name.append(response.country.name)
    lat.append(response.location.latitude)
    lon.append(response.location.longitude)
    

geoip_enrich_IPs_stacked['alpha2_code'] = alpha2    
geoip_enrich_IPs_stacked['alpha3_code'] = alpha3
geoip_enrich_IPs_stacked['country_name'] = country_name
geoip_enrich_IPs_stacked['lat'] = lat
geoip_enrich_IPs_stacked['lon'] = lon

display(geoip_enrich_IPs_stacked.head(20))

#Creates new DF with sum of clientIPs per Alpha_3 ISO code, this is the format for scatter_geo map
aggregateCountry = pd.DataFrame(geoip_enrich_IPs_stacked.drop(['country_name','alpha2_code','lat','lon'], axis=1))
aggregateCountry = aggregateCountry.groupby(['alpha3_code','source_IP']).sum().groupby('alpha3_code').sum()
aggregateCountry = aggregateCountry.stack().reset_index().drop('level_1', axis=1) #THIS gets the job done, but not neat....                               
aggregateCountry.rename(columns={0:'sum'}, inplace=True)

#Do ratios of US vs. non-US
#Visualize traffic by country breakdown with pie chart
fig2 = px.pie(aggregateCountry, values='sum', names='alpha3_code', title='Percentage of Source IP Traffic Observed by Country')
fig2.update_traces(textposition='inside', textinfo='percent+label')
fig2.show()

#Uncomment if you need to save out image!
output_path_png = str(images_folder)+'/ip_country_pieChart.png'
output_path_html = str(images_folder)+'/ip_country_pieChart.html'

try:
    fig2.write_image(output_path_png)
    fig2.write_html(output_path_html)

except ValueError as con:
    print('Connection refused error to orca')

else:
    print('Image was saved out as "ip_country_pieChart" in both html and png format!')

In [ ]:
#Quick long tail check
geoip_enrich_IPs_stacked.tail(20)

## User-Agent String Enrichment Pivot

### User-Agent Stack Count
Use this to grab top-talker - Amazon Health Check - and the filter that out of the dataframe.

In [ ]:
apache_drop['user_agent'].value_counts()

In [ ]:
user_agent_df = apache_drop.copy()

In [ ]:
len(user_agent_df)

In [ ]:
user_agent_df = user_agent_df[~(user_agent_df['user_agent'] == 'Amazon-Route53-Health-Check-Service (ref afa8566f-3fc3-4314-9bad-cfe2fb9dba91; report http://amzn.to/1vsZADi)')] 

In [ ]:
len(user_agent_df)

### User-Agent Parsing Pivot

In [ ]:
aggregateUAs = pd.DataFrame(user_agent_df.user_agent.value_counts().reset_index().values, columns=["userAgent", "count"])
ua_head_var = 20
print('The top',ua_head_var,'User-Agent strings from stack counting, after filtering out AWS Health Check, are shown below:')
display(aggregateUAs.head(ua_head_var))

In [ ]:
#Iterate over UA values in column for follow-on pivot/parse

#Create lists to hold the new series data
user_parsed = []
is_mobile = []
is_bot = []

#Was recommended to not do an apply in this multi-column scenario...go figure!
for ua in aggregateUAs['userAgent']:
    user_agent = parse(ua)
    user_parsed.append(str(user_agent))
    is_mobile.append(user_agent.is_mobile)
    is_bot.append(user_agent.is_bot)

aggregateUAs['userAgent-parsed']=user_parsed
aggregateUAs['is-mobile']=is_mobile
aggregateUAs['is-bot']=is_bot

#ua_head_var passed in from previous cell!
print('The top',ua_head_var,'User-Agent strings from stack counting, after the AWS Health Check was filtered, are shown below:')
display(aggregateUAs.head(ua_head_var))

#Write results to file
with open (output_folder/"user_agent_count_enrich.csv", "w") as outfile:
    aggregateUAs.to_csv(outfile, index=False)

print('DataFrame was saved out as "user_agent_count_enrich.csv"')

#Determine ratios for mobile vs non-mobile    
#Determine ratios for bot vs. non-bot
aggregateUAs_copy = aggregateUAs.copy()
trueMobile = aggregateUAs_copy[aggregateUAs_copy['is-mobile'] == True]
trueBots = aggregateUAs_copy[aggregateUAs_copy['is-bot'] == True]

def percentage(part, whole):
    num = 100 * float(part)/float(whole)
    return round(num,2)

print("\n")
print('Total UAs observed: ', aggregateUAs['count'].sum())
print('UAs determined mobile: ', trueMobile['count'].sum())
print('Mobile traffic is',percentage(trueMobile['count'].sum(),aggregateUAs['count'].sum()) ,'% of the overall traffic')
print("\n")
print('Total UAs observed: ', aggregateUAs['count'].sum())
print('UAs determined bot: ', trueBots['count'].sum())
print('Bot traffic is',percentage(trueBots['count'].sum(),aggregateUAs['count'].sum()) ,'% of the overall traffic')

#Visualize UA breakdown with pie chart
fig5 = px.pie(aggregateUAs, values='count', names='userAgent-parsed', title='Percentage of User-Agent Traffic - Post Forward Filter')
fig5.update_traces(textposition='inside', textinfo='percent+label')
fig5.show()

output_path_png = str(images_folder)+'/ua_pieChart.png'
output_path_html = str(images_folder)+'/ua_pieChart.html'

try:
    fig5.write_image(output_path_png)
    fig5.write_html(output_path_html)

except ValueError as con:
    print('Connection refused error to orca')

else:
    print('Image was saved out as "ua_pieChart" in both html and png format!')